# 1. Setting up

Import all required modules

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

np.random.seed(416)

**Note:** Whenever you see print statements, you can uncomment them and run the code block to gain an understanding of the dataset.


Read in the crime data of Philedelphia and view it. For additional info on the dataset we're using, see [this](https://www.kaggle.com/datasets/minnieliang/philadelphia-crime-rate-data).

**Discussion:** Notice the shape of the data. Do you see anything that is a problem?

Let's try to understand the dataset we are using. Each row represents the data of a particular city. the features are:

1. HousePrice = the mean house price in that city
2. HsPrc($10,000) = HousePrice / 10,000
3. CrimeRate = Crime rate in that city
4. MilesPhila = Miles of city from Philedelphia
5. popChg = change in population 
6. Name = Name of the city
7. County = County the city is in

In [2]:
# TODO: read in crime data and view
crime = pd.read_csv("Philadelphia_Crime_Rate_noNA.csv")
crime.head()

,HousePrice,"HsPrc ($10,000)",CrimeRate,MilesPhila,PopChg,Name,County
0,140463,14.0463,29.7,10.0,-1.0,Abington,Montgome
1,113033,11.3033,24.1,18.0,4.0,Ambler,Montgome
2,124186,12.4186,19.5,25.0,8.0,Aston,Delaware
3,110490,11.0490,49.4,25.0,2.7,Bensalem,Bucks
4,79124,7.9124,54.1,19.0,3.9,Bristol B.,Bucks


# 2. Pre-processing

We want to predict the crime rate for a city in Philedelphia. **What input columns should we use?** (There are multiple reasonable answers.)

We believe that the County names are also useful. But they are not numeric.

**Discussion:** How do you think we can use those values? [Hint: Can we numerically encode them?]

In [3]:
#One hot encoding all the county values
one_hot = pd.get_dummies(crime['County'])
#print(one_hot)

#We then concatinate the one hot columns with original dataset.
crime = pd.concat([crime, one_hot], axis=1)

# We need to drop any and all NaN values to filter our data. So we drop any rows that contain a NaN 
crime = crime.dropna()
print(crime)

#Now, let's try to find relevant features in our dataset
input_cols = ["HousePrice", "PopChg", "MilesPhila", 'Bucks', 'Chester', 'Delaware', 'Montgome', 'Phila']
output_col = 'CrimeRate'

    HousePrice  HsPrc ($10,000)  CrimeRate  MilesPhila  PopChg        Name  \
0       140463          14.0463       29.7        10.0    -1.0    Abington   
1       113033          11.3033       24.1        18.0     4.0      Ambler   
2       124186          12.4186       19.5        25.0     8.0       Aston   
3       110490          11.0490       49.4        25.0     2.7    Bensalem   
4        79124           7.9124       54.1        19.0     3.9  Bristol B.   
..         ...              ...        ...         ...     ...         ...   
94      174232          17.4232       13.8        25.0     4.7    Westtown   
95      196515          19.6515       29.9        16.0     1.8  Whitemarsh   
96      232714          23.2714        9.9        21.0     0.2  Willistown   
97      245920          24.5920       22.6        10.0     0.3   Wynnewood   
98      130953          13.0953       13.0        24.0     5.2     Yardley   

      County  Bucks  Chester  Delaware  Montgome  Phila  
0   M

Split the crime data into a training and test set and then store the input and target data seperately for each set. Use [train/test split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from sklearn.

In [4]:
# TODO: split into training and testing sets
train, test = train_test_split(crime, test_size=0.2)

# TODO: split data sets into input and target variables
train_X = train[input_cols]
train_y = train[output_col]

test_X = test[input_cols]
test_y = test[output_col]

Check the shape of each set to make sure they make sense!

In [5]:
print("train input data shape:", train_X.shape)
print("train target data shape:", train_y.shape)
print()
print("test input data shape:", test_X.shape)
print("test target data shape:", test_y.shape)

train input data shape: (78, 8)
train target data shape: (78,)

test input data shape: (20, 8)
test target data shape: (20,)


Normalize training and test set input data (X) using statistics generated from the training set. To do this, use the [Standard Scaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) from sklearn. (**Conceptual Check**: Why is it important to use statistics generated from the training set?)

In [6]:
scaler = StandardScaler()

# TODO: Fit StandardScaler() with training data and apply to both training and test data
scaler.fit(train_X)
train_X_norm = scaler.transform(train_X)
test_X_norm = scaler.transform(test_X)

View the type of the data post-normalization (as well as the data itself).

In [7]:
print("data type after normalizaton:", type(train_X_norm))
pd.DataFrame(train_X_norm)

data type after normalizaton: <class 'numpy.ndarray'>


,0,1,2,3,4,5,6,7
0,0.447448,-0.906123,0.347822,-0.467707,-0.467707,-0.587220,1.546384,-0.313993
1,0.510510,-1.216501,0.117418,-0.467707,-0.467707,-0.587220,1.546384,-0.313993
2,-0.692668,0.623599,-2.186629,-0.467707,-0.467707,-0.587220,-0.646670,3.184785
3,-0.424976,0.823128,0.693429,-0.467707,-0.467707,-0.587220,1.546384,-0.313993
4,-0.228859,0.490580,1.384643,2.138090,-0.467707,-0.587220,-0.646670,-0.313993
...,...,...,...,...,...,...,...,...
73,-0.487978,0.446240,-0.112987,-0.467707,-0.467707,-0.587220,1.546384,-0.313993
74,0.575603,1.998131,1.960655,-0.467707,2.138090,-0.587220,-0.646670,-0.313993
75,-1.351744,-1.704238,-1.725820,-0.467707,-0.467707,-0.587220,-0.646670,3.184785
76,-0.777229,-1.016972,-0.458594,-0.467707,-0.467707,1.702939,-0.646670,-0.313993


# 3. Regularization with Ridge

Create a [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) linear model with a regularization coefficent of 1. 

Note: This coefficent is referred to as "lambda (λ)" in course material and "alpha" in the sklearn docs. They are the same thing!

In [8]:
# TODO: construct Ridge regularization model with alpha=1.0
ridge_model = Ridge(alpha=1.0)

Train the model using the training data and output the training error. To do so, define a function rmse(mode, X, y) that calculates the RMSE error for a given model, input, and target data.

In [9]:
def rmse(model, X, y):
    predictions = model.predict(X)
    return mean_squared_error(predictions, y, squared=False)

In [10]:
# TODO: train Ridge model with training data and output train error using rmse()
ridge_model.fit(train_X_norm, train_y)
rmse(ridge_model, train_X_norm, train_y)

c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


np.float64(32.91777348432889)

Perform 5-fold cross validation with your Ridge model. Output the array of errors (length 5) as well as the mean error. You should use [Cross Validation Score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html?highlight=cross_val_scor) from sklearn to do this.

In [11]:
# TODO: fill out parameters for cross_val_score() and print errors
ridge_CV_scores = cross_val_score(ridge_model, train_X_norm, train_y, cv=5, scoring=rmse)

c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarnin

Perform 5-fold cross validation on Ridge models with a range of alpha values. For each alpha, print the alpha value and the corresponding mean CV score.

In [12]:
for reg_coef in [0.0001, 0.1, 1, 10, 100, 1000, 10e4, 10e7]:
    ridge_model = Ridge(alpha=reg_coef)
    ridge_CV_scores = cross_val_score(ridge_model, train_X_norm, train_y, cv=5, scoring=rmse)
    print(reg_coef, ridge_CV_scores.mean(), sep='\t')

0.0001	37.50090263606724
0.1	37.43970007336746
1	36.91730282456719
10	33.56965319795065
100	29.62088364376266
1000	31.26781831761904
100000.0	32.008549856873906
100000000.0	32.017569248632505


c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarnin

Take a look at how the weights of Ridge models change as you change the regularization coefficient!

In [13]:
print("Reg coeff. | ", "Intercept | ", input_cols)
print("_________________________________________________")

for reg_coef in [0.0001, 0.1, 1, 10, 100, 1000, 10e4, 10e7]:
    ridge_model = Ridge(alpha=reg_coef)
    ridge_model.fit(train_X_norm, train_y)
    print(reg_coef," | ", ridge_model.intercept_, " | ", ridge_model.coef_)
    print()

Reg coeff. |  Intercept |  ['HousePrice', 'PopChg', 'MilesPhila', 'Bucks', 'Chester', 'Delaware', 'Montgome', 'Phila']
_________________________________________________
0.0001  |  33.57435897435898  |  [ -3.63197189  17.13681315 -14.24752441  -2.23370832  -2.70466811
  -7.72951492   0.04520311  18.36731872]

0.1  |  33.57435897435898  |  [ -3.63608447  17.07375856 -14.19995246  -2.23448001  -2.69413453
  -7.71463392   0.03962798  18.34037168]

1  |  33.57435897435898  |  [-3.67137431e+00  1.65239914e+01 -1.37880362e+01 -2.23958735e+00
 -2.60158983e+00 -7.58471991e+00 -8.38937656e-03  1.81011014e+01]

10  |  33.57435897435898  |  [ -3.88521252  12.38453933 -10.83241873  -2.18627031  -1.87616968
  -6.57512436  -0.33331877  16.03148365]

100  |  33.57435897435898  |  [-3.10333377  2.75480833 -4.24249064 -1.31121125 -0.3460951  -3.10006416
 -0.64721913  7.99392311]

1000  |  33.57435897435898  |  [-0.70776377  0.090652   -0.79471316 -0.28442008 -0.07301084 -0.47373972
 -0.16754507  1.47097

**Concept check**: How would the weights be different if you didn't regularize them? (i.e., use `LinearRegression` instead of `Ridge`.)


In [14]:
Larger

NameError: name 'Larger' is not defined

# 4. Regularization with LASSO

Create a [LASSO](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) linear model with a regularization coefficent of 1.

In [ ]:
# TODO: construct LASSO regularization model with alpha=1.0
lasso_model = Lasso(alpha=1.0)

Train the model using the training data and output the training error.

In [ ]:
# TODO: train LASSO model with training data and output train error using rmse()# TODO: train LASSO model with training data and output train error using rmse()
lasso_model.fit(train_X_norm, train_y)
rmse(lasso_model, train_X_norm, train_y)

c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


np.float64(33.06978156973761)

Perform 5-fold cross validation with your LASSO model. Output the array of errors (length 5) as well as the mean error.

In [ ]:
# TODO: fill out parameters for cross_val_score() and print errors
lasso_CV_scores = cross_val_score(lasso_model, train_X_norm, train_y, cv=5, scoring=rmse)
print(lasso_CV_scores)
print(lasso_CV_scores.mean())

[79.78017559 30.04003674 24.7154594  27.98819809 16.13101237]
35.73097643999485


c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarnin

Perform 5-fold cross validation on LASSO models with a range of alpha values. For each alpha, print the alpha value and the corresponding mean CV score.

In [ ]:
for reg_coef in [1e-07,1e-05, 0.001, 0.1, 1, 10, 100, 1000]:
    lasso_model = Lasso(alpha=reg_coef)
    lasso_CV_scores = cross_val_score(lasso_model, train_X_norm, train_y, cv=5, scoring=rmse)
    print(reg_coef, lasso_CV_scores.mean(), sep='\t')

c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.148e+02, tolerance: 1.975e+00
  model = cd_fast.enet_coordinate_descent(
c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\achar\AppData\Local\Pro

1e-07	37.50096415041345
1e-05	37.500955221160744
0.001	37.50025748577373
0.1	37.340701781518014
1	35.73097643999485
10	30.77789845089668
100	32.01757829516642
1000	32.01757829516642


Take a look at how the weights of LASSO models change as you change the regularization coefficient!

Note: In python, -0 is the same as 0!

In [ ]:
print("Reg coeff. | ", "Intercept | ", input_cols)
print("_________________________________________________")
print()

for reg_coef in [1e-07,1e-05, 0.001, 0.1, 1, 10, 100, 1000]:
    lasso_model = Lasso(alpha=reg_coef)
    lasso_model.fit(train_X_norm, train_y)
    print(reg_coef, " | ", lasso_model.intercept_, " | ",  lasso_model.coef_)
    print()

Reg coeff. |  Intercept |  ['HousePrice', 'PopChg', 'MilesPhila', 'Bucks', 'Chester', 'Delaware', 'Montgome', 'Phila']
_________________________________________________

1e-07  |  33.57435897435898  |  [ -3.63196768  17.1368763  -14.24757218  -5.6127896   -6.08376072
 -11.57432963  -3.96983238  15.85069499]

1e-05  |  33.57435897435898  |  [ -3.63196065  17.13684905 -14.24754555  -5.61242715  -6.08339121
 -11.57390263  -3.96939208  15.85096386]

0.001  |  33.57435897435898  |  [ -3.63125564  17.13412272 -14.24483737  -5.58298541  -6.053239
 -11.53891058  -3.93342704  15.87281536]

0.1  |  33.57435897435898  |  [ -3.54024151  16.79550301 -13.9516437    0.          -0.31594044
  -4.99649311   2.64377887  20.00171027]

1  |  33.57435897435898  |  [ -2.70661526  13.90488014 -11.36208519  -0.          -0.
  -3.85457208   1.55060947  19.39796805]

10  |  33.57435897435898  |  [-0.          0.         -0.         -0.          0.         -0.
 -0.         11.08146116]

100  |  33.57435897435898

**Discussion:** Do you notice anything interesting about the weights overall? Do you notice anything surprising about the weights of the (first) best performing alpha?

# 5. Computing final test scores

Using the regularization coefficient that leads to the best validation error, compute test scores for a Ridge and LASSO model.

In [ ]:
# TODO: choose best alphas from above and calculate test errors
print("Ridge", rmse(Ridge(alpha=1.0).fit(train_X_norm, train_y), test_X_norm, test_y))
print("LASSO", rmse(Lasso(alpha=1.0).fit(train_X_norm, train_y), test_X_norm, test_y))
print("LinearRegression", rmse(LinearRegression().fit(train_X_norm, train_y), test_X_norm, test_y))

Ridge 31.37520198091263
LASSO 27.302912339711856
LinearRegression 32.326371126535946


c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Now, let's use the same models on the unnormalized training set.

In [15]:
print("Ridge", rmse(Ridge(alpha=100).fit(train_X, train_y), test_X, test_y))
print("LASSO", rmse(Lasso(alpha=10).fit(train_X, train_y), test_X, test_y))
print("LinearRegression", rmse(LinearRegression().fit(train_X, train_y), test_X, test_y))

Ridge 21.395689707502893
LASSO 16.7213837215884
LinearRegression 32.32637112653585


c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\achar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


**Discussion:** So as you can see, the regularized models perform significantly well as opposed to the simple linear models. However, they put too much weight on the 'Phila' county. If we put this model to use in real world, what can be the consequences? Are there any steps we can take to avoid these misinterpretations?

**Big Takeaway:** Correlation does not imply causality